v0.1

>

In [12]:
import numpy as np
import pandas as pd
import sys
# import polars as pl
import warnings

# import re
# import time

# from copy import copy, deepcopy

In [13]:
IS_GUEST = False
LOAD_SAVED_DATA = False

In [14]:
if not IS_GUEST:
    !if [ ! -f "/content/helpers/clean_sentences.py" ]; then wget -P helpers/ https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/clean_sentences.py; fi

In [17]:
if not IS_GUEST:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    root_dir = "/content/drive/MyDrive/softuni/the-grammar-whisperer"

data_raw_dir = f"{root_dir}/data/raw"
data_clean_dir = f"{root_dir}/data/clean"
helpers_dir = f"{root_dir}/helpers"

if root_dir not in sys.path:
    sys.path.append(root_dir)
    sys.path.append(helpers_dir)

Mounted at /content/drive


In [18]:
from clean_sentences import clean_sentences

ImportError: cannot import name 'clean_sentences' from 'clean_sentences' (/content/drive/MyDrive/softuni/the-grammar-whisperer/helpers/clean_sentences.py)

In [ ]:
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning) # 🧡

## Load raw data containing Bulgarian sentences from fiction literature

TODO:

- [ ] Load raw chunks (8)
- [ ] clean the chunks and save to tsv
- [ ] Concatenate all chunks into 1 df
- [ ] deduplicate and sort the whole df

###### Load all text files with Bulrarian fiction (raw)

In [ ]:
input_raw_texts = [
    "chunk_1.txt",
    "chunk_2.txt",
    # "chunk_3.txt",
    # "chunk_4.txt",
    # "chunk_5.txt",
    # "chunk_6.txt",
    # "chunk_7.txt",
    # "chunk_8.txt",
]

output_split_texts = [
    "bg_fiction_chunk1_split.tsv",
    "bg_fiction_chunk2_split.tsv",
    # "bg_fiction_chunk3_split.tsv",
    # "bg_fiction_chunk4_split.tsv",
    # "bg_fiction_chunk5_split.tsv",
    # "bg_fiction_chunk6_split.tsv",
    # "bg_fiction_chunk7_split.tsv",
    # "bg_fiction_chunk8_split.tsv",
]

In [ ]:
for raw_text, split_text in zip(input_raw_texts, output_split_texts):
    bg_text_path = f"{data_raw_dir}\\{raw_text}"
    output_path = f"{data_clean_dir}/{split_text}"
    sentences = split_and_save_sentences(bg_text_path, output_path)

In [1]:
pppppp

NameError: name 'pppppp' is not defined

##### Load csv and split

In [ ]:
df = pd.read_csv(wl_wiki_2021_1M, sep='\t', names=['seq', 'sentence'])

##### Remove unneeded columns

In [ ]:
if 'seq' in df.columns:
    df = df.drop(columns=['seq'])

##### Replace hypens, whitespaces, quotes

In [ ]:
df.shape

(938240, 1)

In [ ]:
hyphen_chars = ['\u2010','\u2011','\u2012','\u2013','\u2014','\u2015','\u2212','\u2043','\uFE58','\uFE63','\uFF0D' ]
whitespace_chars = ' \xa0\u1680\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u2028\u2029\u202f\u205f\u3000\t\n\r\v\f'
doubleqoute_chars = ['\u201c','\u201d','\u2033','\u201e','\u201c','\u00ab','\u00bb',]
singlequote_chars = ['\u2032','\u2018','\u2019','\u0060']

for h in hyphen_chars:
    df['sentence'] = df['sentence'].str.replace(h, '-')

for w in whitespace_chars:
    df['sentence'] = df['sentence'].str.replace(w, ' ')

for q in doubleqoute_chars:
    df['sentence'] = df['sentence'].str.replace(q, '"')

for q in singlequote_chars:
    df['sentence'] = df['sentence'].str.replace(q, '"')

# df['sentence'] = df['sentence'].str.replace('\n', '')
# df['sentence'] = df['sentence'].str.replace('\r', '')
df['sentence'] = df['sentence'].str.strip('[ \t]').str.strip()


##### Remove fows with forbidden characters

In [ ]:
allowed_pattern = '[^а-яА-Яъѝю0-9\,.!?":;… \\\/\'\(\)\–\-]'

# Extract rows containing any characters outside the allowed set
df_invalid = df[df['sentence'].str.contains(allowed_pattern, regex=True)]
df_valid = df[~df['sentence'].str.contains(allowed_pattern, regex=True)]

print("Original # rows:", df.shape)
print("Invalid # rows: ", df_invalid.shape)
print("Valid # rows:   ", df_valid.shape)

Original # rows: (938240, 1)
Invalid # rows:  (115217, 1)
Valid # rows:    (823023, 1)


In [ ]:
assert df_valid.shape[0] == 823023 # 💙better assert

In [ ]:
df = df_valid

##### Remove long sentences

In [ ]:
df.shape

(823023, 1)

In [ ]:
# Remove rows where the length of the 'sentence' column is greater than 250 characters
df = df[df['sentence'].str.len() <= 250]
df.shape

(821427, 1)

##### Remove rows not starting with a letter

In [ ]:
# Function to check if a string starts with a Cyrillic letter
def starts_with_cyrillic(text):
    if not isinstance(text, str):  #Handle cases where sentence might not be a string
      return False
    return bool(re.match(r'^[а-яА-ЯъѝюЪЮ]', text))

# Filter the DataFrame to keep only rows where the 'sentence' column starts with a Cyrillic letter
df = df[df['sentence'].apply(starts_with_cyrillic)]

# Now df_valid contains only the rows that meet the specified criteria.
df.shape

(812582, 1)

##### Remove rows not ending in punctuation mark

In [ ]:
df.shape

(812582, 1)

In [ ]:
# Function to check if a sentence ends with a punctuation mark, with an exception for double quotes
def ends_with_punctuation(sentence):
    if not isinstance(sentence, str):
        return False
    match = re.search(r'([.!?;…])\s*"$', sentence)
    if match:
        return True

    # match = re.search(r'([.!?;…])\s*\'$', sentence)
    # if match:
    #     return True
    return bool(re.search(r'[.!?;…]$', sentence))

# Create a boolean mask for rows that end with punctuation or the double quote exception
mask = df['sentence'].apply(ends_with_punctuation)

# Create the new DataFrame with invalid rows
df_invalid = df[~mask]

# Remove invalid rows from the original DataFrame
df_valid = df[mask]

print("Valid sentences after punctuation check:", df_valid.shape)
print("Invalid sentences:", df_invalid.shape)

Valid sentences after punctuation check: (787245, 1)
Invalid sentences: (25337, 1)


In [ ]:
df_valid.shape

(787245, 1)

In [ ]:
for x in df_valid.iloc[100:200]['sentence']:
    print(x)

Абасуба са сериозно повлияни от културата на луо чрез контакти и смесени бракове.
Абасуба са силно повлияни от доминиращите на острова луо чрез контакти и смесени бракове.
Абат Абон ги посреща подобаващо в манастира.
Абатите заемат различно положение по отношение на ордена, от една страна, и по отношение подчинените им монаси, от друга страна.
Абатите на Дюлюгем предприемат мащабно строителство и подобрения на съществуващите сгради в неокласически стил, по проект на архитекта на кралските дворци и обществени сгради, Лоран Беноа Деюс (1731 - 1812), което продължава от 1775 до 1791 г.
Абат-Йерусалем-монумент в дворцовата градина Фехелде.
Абат на Бенедиктския манастир край Ферие.
Абат от 1996 г. е Джакомо Бриере.
Абатския дворец проектиран от архитекта Лоран Беноа Деюс единствената оцеляла до наши дни сграда от манастирския комплекс.
Абатският дворец е център за провеждане на изложби, семинари и други културни мероприятия.
Абатства и епархии остават измамени от всяко завещание, което няко

In [ ]:
df = df_valid

##### Remove empty braces

In [ ]:
def clean_empty_braces(sentence):
    # Regular expression to match brackets with optional whitespace inside
    pattern = r"\(\s*\)"
    cleaned_sentence = re.sub(pattern, "", sentence)
    # Collapse multiple whitespaces into single whitespace
    cleaned_sentence = re.sub(r"\s+", " ", cleaned_sentence).strip()
    return cleaned_sentence

df['sentence'] = df['sentence'].apply(clean_empty_braces)

In [ ]:
df.shape, df_valid.shape

((787245, 1), (787245, 1))

##### Shuffle all rows

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
for x in df.iloc[100:200]['sentence']:
    print(x)

Член е на Съюза на писателите в Естония от 2012 г.
В такъв случай се препоръчва третиране с фунгицид и бактерициден препарат на медна основа.
При стареенето на клетките тези рибозоми могат отново да преминат в свободно състояние.
Друг важен акцент във филмовото произведение е липсата на човечност у подпоручика Алтънов (изважда пистолет на Палазов, когато той се застъпва за г-ца Барбиери), фелдшера Марински и кръчмаря Сандьо, изразено чрез сцената на лов за птици.
Четата му пристига в Прилепско, за да екзекутира няколко местни дерибеи, начело с Расим от Десово.
Централната част на триптиха представлява "Дева Мария в слава" /"Коронясването на Богородица"/.
Първоначално гозбата е чисто местен специалитет, който чак през 20 век добива известност извън региона на Ница.
След създаването на гръцката държава е принуден да се върне в Битоля, тъй като е отхвърлен като чужденец.
Всеки може да се занимава с фрийрън, защото това е спорт за здраве, стига да има добра физическа подготовка и хъс.
Чест

##### Save to 2 CSVs
> so that size < 100MB

In [ ]:
n_rows = df.shape[0]

nrows_per_part = n_rows // 2

# split into 2 parts
df_parts = [df.iloc[i*nrows_per_part : (i+1)*nrows_per_part] for i in range(2)]

# 💙if file with that name exists, add _1
# save the resulting DataFrames
for i, part in enumerate(df_parts, 1):
    part.to_csv(f'{data_clean_dir}/wl_bul_wikipedia_2021_1M-sentences_part{i}.csv', index=False, sep='|')